In [1]:
import folium
from src.apikeys import MONGO
from src import process_tweets
from pymongo import MongoClient
import pandas as pd

In [2]:
tweet_processor = process_tweets.TweetProcessor('../models/tfidf_logistic_reg.pk')


In [3]:
client = MongoClient('mongodb://{}:{}@localhost:27017'.format(MONGO.USER, MONGO.PASSWORD))

In [4]:
database = client['twitter_database']

In [5]:
database.collection_names()

['test',
 'trends_2017_10_18_9',
 'trends_2017_10_16_17',
 'trends_2017_10_17_18',
 'broncos_raiders',
 'available_trends',
 'test_method',
 'trends_2017_10_17_19',
 'patriots_panthers',
 'trends_2017_10_17_17',
 'roll_tide']

In [6]:
table = database['broncos_raiders']

In [7]:
df = tweet_processor.process_database(table, topics=['broncos'])

In [8]:
df.head(10)

,text,state,date,sentiment,time_bin
0,Thought ya learned from the Broncos game but g...,Texas,2017-10-01 20:23:18+00:00,0.548985,0
1,@codycoil I don't remind u when the broncos lo...,None,2017-10-01 20:23:18+00:00,0.247567,0
2,@MitzSumilang If my BRONCOS kneel....I'm shutt...,Colorado,2017-10-01 20:23:19+00:00,0.491284,0
3,RT @Broncos: About that time.\n\n#BeatTheRaide...,None,2017-10-01 20:23:19+00:00,0.188382,0
4,RT @957thegame: #Raiders win toss and defer. #...,Florida,2017-10-01 20:23:20+00:00,0.491284,0
5,RT @Broncos: #BeatTheRaiders https://t.co/ATCI...,California,2017-10-01 20:23:20+00:00,0.491284,0
6,RT @Broncos: Nothing better than a rivalry gam...,None,2017-10-01 20:23:20+00:00,0.123702,0
7,RT @WKrummholz: @DenverChannel @Broncos Messag...,None,2017-10-01 20:23:23+00:00,0.491284,0
8,RT @NickiJhabvala: Broncos' inactives \nQB Pax...,Colorado,2017-10-01 20:23:23+00:00,0.491284,0
9,RT @raidersFGSN: RAIDERS VS BRONCOS IN 10 minu...,None,2017-10-01 20:23:24+00:00,0.973583,0


In [25]:
from src.visualization import visualize_sentiment
from src.visualization import visualize_count
my_map = visualize_sentiment(df)
my_map

In [26]:
# my_map.save('../maps/test_map.html')

In [29]:
import numpy as np
a = df.groupby('state')['sentiment'].agg(['mean','std','count'])
a['mean_dev'] = a.apply(lambda row: row['std']/np.sqrt(row['count']),axis=1)
a[['mean','mean_dev']].head(50)

,mean,mean_dev
state,,
Alabama,0.529964,0.020056
Alaska,0.523932,0.022371
Arizona,0.548932,0.009007
Arkansas,0.535114,0.022808
California,0.517887,0.003698
Colorado,0.524025,0.002996
Connecticut,0.530023,0.014884
Delaware,0.513343,0.009422
District of Columbia,0.548497,0.035539


In [28]:
my_map, avg_sentiment = visualize_count(df)
print(avg_sentiment)
my_map

0.5237527403035627


In [12]:
for i in df[df['sentiment']>.8]['text'].head(30):
    print(i)

RT @raidersFGSN: RAIDERS VS BRONCOS IN 10 minutes! Who's ready for our W in Denver!?! 🤞🏽🤞🏽🤞🏽#raiders #OAKvsDEN
@FoxNews @MikeTobinFox @ANHQDC The name of the game is 'wait in line and hoard all you can,'" @RAIDERS vs @Broncos… https://t.co/vLVxUut9ty
RT @natelundy: FYI that Thunder took a crap all over the field on the run out of the tunnel. 

So take that however you'd like. 

#Broncos
@vitaminZD Y’all are not good you got ya a$$ kicked by the broncos
RT @WKrummholz: @Rampage95x2 Great video. I am done with the Broncos also. 🇺🇸 #BoycottNFL
#Broncos provided the layout to beat Dallas. And of course Wade Phillip's Rams D followed along. 😊 Good win, Rams!
@NFL_Memes Today was a good day! @RAIDERS vs @Broncos #get_free_access https://t.co/ELWPo1JvMx
Tequila Drinks We Love: The Broncos Sunset https://t.co/IJQvye6rwq via @liquor.com
Omg gonna miss the beginning of the broncos game because the trash Jags and Jets wanna tie
Thank you @Broncos beautiful anthem today.
RT @WKrummholz: @Rampage95